## Inserting data in MySQL using Python

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations [https://data.cityofnewyork.us/NYC-BigApps/Citi-Bike-Live-Station-Feed-JSON-/p94q-8hxh](https://data.cityofnewyork.us/NYC-BigApps/Citi-Bike-Live-Station-Feed-JSON-/p94q-8hxh), using the correspoding API call provided by the Citibike [website](https://www.citibikenyc.com/system-data):

In [1]:
import requests
import json

In [2]:
# Let's get the data from the Citibike API
url = 'http://www.citibikenyc.com/stations/json'
resp = requests.get(url)

In [3]:
# We transform the returned JSON answer from the API into a Python dictionary object
results = json.loads(resp.text)

In [4]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only we need
data = results["stationBeanList"]

In [5]:
len(data)

814

In [6]:
#print an example
print(data[0])

{'id': 281, 'stationName': 'Grand Army Plaza & Central Park S', 'availableDocks': 36, 'totalDocks': 57, 'latitude': 40.7643971, 'longitude': -73.97371465, 'statusValue': 'In Service', 'statusKey': 1, 'availableBikes': 18, 'stAddress1': 'Grand Army Plaza & Central Park S', 'stAddress2': '', 'city': '', 'postalCode': '', 'location': '', 'altitude': '', 'testStation': False, 'lastCommunicationTime': '2018-11-15 04:35:46 PM', 'landMark': ''}


Now we will connect to our MySQL server. We will use the `pymysql` library of Python.

If you do not have the library, you need to install it by typing in the shell:

`conda install pymysql`

In [7]:
import pymysql
import sys

connection = pymysql.connect(host = 'localhost',
                             user = 'root',
                             passwd = '',
                             charset='utf8', use_unicode=True);

Once we have connected successfully, we need to create our database:

In [8]:
# Query to create a database
db_name = 'citibike'
create_db_query = "CREATE DATABASE IF NOT EXISTS {0} DEFAULT CHARACTER SET 'utf8'".format(db_name)

# Create a database
cursor = connection.cursor()
cursor.execute(create_db_query)
cursor.close()

Then we create the table where we will store our data. For our example, we will just import three fields in the database: station_id, station_name, and number_of_docks

In [9]:
cursor = connection.cursor()
db_name = 'citibike'
table_name = 'Docks'
# Create a table
# The {0} and {1} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                (station_id int, 
                                station_name varchar(250), 
                                number_of_docks int,
                                available_docks int,
                                date datetime,
                                PRIMARY KEY(station_id, date)
                                )'''.format(db_name, table_name)
cursor.execute(create_table_query)
cursor.close()

Finally, we import the data into our table, using the INSERT command. 

In [10]:
from datetime import date, datetime, timedelta

query_template = '''
INSERT INTO 
citibike.Docks(station_id, station_name,
                number_of_docks, available_docks, date) 
        VALUES (%s, %s, %s, %s, %s)'''

cursor = connection.cursor()

for entry in data:
    dockid = entry["id"]
    addr = entry["stationName"]
    docks = entry["totalDocks"]
    available = entry["availableDocks"]
    # date =  datetime.now()
    # lastcommunicationtime is a string of 
    # the form "2016-02-09 10:16:49 AM"
    # See https://docs.python.org/3.6/library/datetime.html#strftime-and-strptime-behavior
    # to see the documentation on how to parse 
    date = datetime.strptime(entry["lastCommunicationTime"], 
                             '%Y-%m-%d %I:%M:%S %p')
    print("Inserting station", dockid, "at", addr)
    query_parameters = (dockid, addr, docks, available, date)
    cursor.execute(query_template, query_parameters)

'''
Commit changes to stable storage
'''
connection.commit()
cursor.close()

Inserting station 281 at Grand Army Plaza & Central Park S
Inserting station 304 at Broadway & Battery Pl
Inserting station 377 at 6 Ave & Canal St
Inserting station 402 at Broadway & E 22 St
Inserting station 491 at E 24 St & Park Ave S
Inserting station 514 at 12 Ave & W 40 St
Inserting station 520 at W 52 St & 5 Ave
Inserting station 3236 at W 42 St & Dyer Ave
Inserting station 3255 at 8 Ave & W 31 St
Inserting station 3443 at W 52 St & 6 Ave
Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 143 at Clinton St & Joralemon St
Inserting station 144 at Nassau St & Navy St
Inserting station 146 at Hudson St & Reade St
Inserting station

Inserting station 3054 at Greene Ave & Throop Ave
Inserting station 3055 at Greene Ave & Nostrand Ave
Inserting station 3056 at Kosciuszko St & Nostrand Ave
Inserting station 3057 at Kosciuszko St & Tompkins Ave
Inserting station 3058 at Lewis Ave & Kosciuszko St
Inserting station 3059 at Pulaski St & Marcus Garvey Blvd
Inserting station 3060 at Willoughby Ave & Tompkins Ave
Inserting station 3061 at Throop Ave & Myrtle Ave
Inserting station 3062 at Myrtle Ave & Marcy Ave
Inserting station 3063 at Nostrand Ave & Myrtle Ave
Inserting station 3064 at Myrtle Ave & Lewis Ave
Inserting station 3065 at Union Ave & Wallabout St
Inserting station 3066 at Tompkins Ave & Hopkins St
Inserting station 3067 at Broadway & Whipple St
Inserting station 3068 at Humboldt St & Varet St
Inserting station 3069 at Lorimer St & Broadway
Inserting station 3070 at McKibbin St & Manhattan Ave
Inserting station 3071 at Boerum St & Broadway
Inserting station 3072 at Leonard St & Boerum St
Inserting station 3074 a

Inserting station 3429 at Hanson Pl & Ashland Pl
Inserting station 3430 at Richardson St & N Henry St
Inserting station 3431 at E 35 St & 3 Ave
Inserting station 3434 at W 88 St & West End Ave
Inserting station 3435 at Grand St & Elizabeth St
Inserting station 3437 at Riverside Dr & W 91 St
Inserting station 3440 at Fulton St & Adams St
Inserting station 3449 at Eckford St & Engert Ave
Inserting station 3452 at Bayard St & Leonard St
Inserting station 3453 at Devoe St & Lorimer St
Inserting station 3454 at Leonard St & Maujer St
Inserting station 3456 at Jackson St & Leonard St
Inserting station 3457 at E 58 St & Madison Ave
Inserting station 3458 at W 55 St & 6 Ave
Inserting station 3459 at E 53 St & 3 Ave
Inserting station 3461 at Murray St & Greenwich St
Inserting station 3462 at E 44 St & 2 Ave
Inserting station 3463 at E 16 St & Irving Pl
Inserting station 3467 at W Broadway & Spring Street
Inserting station 3469 at India St & West St
Inserting station 3472 at W 15 St & 10 Ave
Ins

In [ ]:
#Check if data in the databse
cursor = connection.cursor(pymysql.cursors.DictCursor)
cursor.execute("SELECT * FROM citibike.Docks")
rows = cursor.fetchall()

for row in rows:
    print("Station ID:", row["station_id"])
    print("Station Name:", row["station_name"])
    print("Number of Docks:", row["number_of_docks"])
    print("Available Docks:", row["available_docks"])
    print("Last Communication:", row["date"])
    print("=============================================")
    
cursor.close()

We can, of course, transform the results back into a DataFrame (see below) or we can use the data directly from the rows object (which is a tuple, containing one dictionary object for each line of the results).

In [12]:
import pandas as pd

cursor = connection.cursor(pymysql.cursors.DictCursor)
cursor.execute("SELECT * FROM citibike.Docks")
rows = cursor.fetchall()
df_from_sql = pd.DataFrame(list(rows))
df_from_sql

,available_docks,date,number_of_docks,station_id,station_name
0,35,2018-11-15 16:37:20,39,72,W 52 St & 11 Ave
1,9,2018-11-15 16:37:03,33,79,Franklin St & W Broadway
2,16,2018-11-15 16:37:08,27,82,St James Pl & Pearl St
3,0,2018-10-09 09:05:09,0,83,Atlantic Ave & Fort Greene Pl
4,11,2018-11-15 16:36:59,19,119,Park Ave & St Edwards St
5,18,2018-11-15 16:35:33,19,120,Lexington Ave & Classon Ave
6,5,2018-11-15 16:35:39,31,127,Barrow St & Hudson St
7,13,2018-11-15 16:34:32,30,128,MacDougal St & Prince St
8,13,2018-11-15 16:34:35,24,143,Clinton St & Joralemon St
9,1,2018-11-15 16:34:41,19,144,Nassau St & Navy St


In [13]:
# We can then compute functions directly on the dataframe
print(sum(df_from_sql["number_of_docks"]))
print(sum(df_from_sql["available_docks"]))

24332
14153


In [14]:
# The other way to compute functions directly on the dataframe
print(df_from_sql["number_of_docks"].sum())
print(df_from_sql["available_docks"].sum())

24332
14153


In [15]:
# And we can also create 
df_from_sql["bikes_taken"] = df_from_sql["number_of_docks"] - df_from_sql["available_docks"]

In [16]:
df_from_sql.head(5)

,available_docks,date,number_of_docks,station_id,station_name,bikes_taken
0,35,2018-11-15 16:37:20,39,72,W 52 St & 11 Ave,4
1,9,2018-11-15 16:37:03,33,79,Franklin St & W Broadway,24
2,16,2018-11-15 16:37:08,27,82,St James Pl & Pearl St,11
3,0,2018-10-09 09:05:09,0,83,Atlantic Ave & Fort Greene Pl,0
4,11,2018-11-15 16:36:59,19,119,Park Ave & St Edwards St,8


Finally, let's close our database connection.

In [17]:
connection.close()

In [18]:
#store data into xls and csv formats
df_from_sql.to_excel("citibike.xls")
df_from_sql.to_csv("citibike.csv")

## Exercise

Create a new table `'Station'` to store stable information about the station, such as location and street address.

In [ ]:
# Your code is here
cursor = connection.cursor()
db_name = 'citibike'
table_name = 'Station'
# Create a table
# The {0} and {1} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                (station_id int, 
                                station_name varchar(250), 
                                number_of_docks int,
                                address varchar(250),
                                PRIMARY KEY(station_id)
                                )'''.format(db_name, table_name)
cursor.execute(create_table_query)
cursor.close()

In [ ]:
query_template = '''
INSERT INTO 
citibike.Station(station_id, station_name,
                number_of_docks, address) 
        VALUES (%s, %s, %s, %s)'''

cursor = connection.cursor()

for entry in data:
    stationid = entry["id"]
    name = entry["stationName"]
    docks = entry["totalDocks"]
    address = entry["stAddress1"]
    print("Inserting station", stationid, "at", address)
    query_parameters = (stationid, name, docks, address)
    cursor.execute(query_template, query_parameters)

'''
Commit changes to stable storage
'''
connection.commit()
cursor.close()